In [310]:
import torchvision


In [311]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(trained = True)

In [312]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [313]:
import numpy as np 
from PIL import Image
import pandas as pd 
import os
import random 
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt 
from torchvision import transforms as T 
import torch 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [314]:
test_df = pd.read_csv("data/labels/test.csv")

In [315]:
test_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6 - C...,640,480,awake,210,201,514,472
1,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6.jpg,640,480,awake,199,194,510,472
2,awake.6d205314-a834-11ee-ba44-e00af6af6ca6 - C...,640,480,awake,236,183,515,471
3,awake.6d205314-a834-11ee-ba44-e00af6af6ca6.jpg,640,480,awake,212,193,509,474
4,awake.6e55e25b-a834-11ee-91f0-e00af6af6ca6 - C...,640,480,awake,236,194,446,457
...,...,...,...,...,...,...,...,...
60,drowsy.8720eaec-a834-11ee-b909-e00af6af6ca6.jpg,640,480,sleepy,282,40,590,471
61,drowsy.8856b595-a834-11ee-a806-e00af6af6ca6.jpg,640,480,sleepy,172,66,502,472
62,drowsy.9950fe22-a834-11ee-a591-e00af6af6ca6.jpg,640,480,awake,216,81,478,455
63,drowsy.94769c9b-a834-11ee-b822-e00af6af6ca6.jpg,640,480,awake,193,86,459,457


In [316]:
test_df = test_df.drop(["width","height",],axis=1)
test_df

,filename,class,xmin,ymin,xmax,ymax
0,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6 - C...,awake,210,201,514,472
1,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6.jpg,awake,199,194,510,472
2,awake.6d205314-a834-11ee-ba44-e00af6af6ca6 - C...,awake,236,183,515,471
3,awake.6d205314-a834-11ee-ba44-e00af6af6ca6.jpg,awake,212,193,509,474
4,awake.6e55e25b-a834-11ee-91f0-e00af6af6ca6 - C...,awake,236,194,446,457
...,...,...,...,...,...,...
60,drowsy.8720eaec-a834-11ee-b909-e00af6af6ca6.jpg,sleepy,282,40,590,471
61,drowsy.8856b595-a834-11ee-a806-e00af6af6ca6.jpg,sleepy,172,66,502,472
62,drowsy.9950fe22-a834-11ee-a591-e00af6af6ca6.jpg,awake,216,81,478,455
63,drowsy.94769c9b-a834-11ee-b822-e00af6af6ca6.jpg,awake,193,86,459,457


In [317]:
torch.tensor(test_df[["xmin","xmax","ymin","ymax"]].iloc[0].to_numpy())
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(["awake","sleepy","background"])

classes = encoder.classes_

test_df["class"] = encoder.transform(test_df["class"])

print(classes)

['awake' 'background' 'sleepy']


In [318]:
classes

array(['awake', 'background', 'sleepy'], dtype='<U10')

In [319]:
test_df

,filename,class,xmin,ymin,xmax,ymax
0,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6 - C...,0,210,201,514,472
1,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6.jpg,0,199,194,510,472
2,awake.6d205314-a834-11ee-ba44-e00af6af6ca6 - C...,0,236,183,515,471
3,awake.6d205314-a834-11ee-ba44-e00af6af6ca6.jpg,0,212,193,509,474
4,awake.6e55e25b-a834-11ee-91f0-e00af6af6ca6 - C...,0,236,194,446,457
...,...,...,...,...,...,...
60,drowsy.8720eaec-a834-11ee-b909-e00af6af6ca6.jpg,2,282,40,590,471
61,drowsy.8856b595-a834-11ee-a806-e00af6af6ca6.jpg,2,172,66,502,472
62,drowsy.9950fe22-a834-11ee-a591-e00af6af6ca6.jpg,0,216,81,478,455
63,drowsy.94769c9b-a834-11ee-b822-e00af6af6ca6.jpg,0,193,86,459,457


In [320]:
labels = torch.tensor(test_df["class"].iloc[0])
labels

tensor(0, dtype=torch.int32)

In [321]:
from torch.utils.data import Dataset, DataLoader


class CusData(Dataset):
    def __init__(self, df,indices):
        super().__init__()
        self.df = df
        self.imgs = df["filename"]
        self.indices = indices

    def __len__(self):
        return len(self.indices)
    def __getitem__(self, index):
         print(self.df["class"].iloc[index])
         image_name = self.df["filename"].iloc[index]
         boxes =torch.tensor(test_df[["xmin","ymin","xmax","ymax"]].iloc[index].to_numpy())
         boxes = boxes.unsqueeze(0)
         img = np.array(Image.open("data/images/"+self.df['filename'].iloc[index]).convert('RGB'))
         img = img.astype('float') /255
         img = torch.tensor(img).permute(2,1,0)
         labels = torch.tensor(self.df["class"].iloc[index])
         target = {}
         target["boxes"] = boxes
         target["labels"] = labels
         return img,target

In [322]:
test_df

,filename,class,xmin,ymin,xmax,ymax
0,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6 - C...,0,210,201,514,472
1,awake.6be7a120-a834-11ee-87ec-e00af6af6ca6.jpg,0,199,194,510,472
2,awake.6d205314-a834-11ee-ba44-e00af6af6ca6 - C...,0,236,183,515,471
3,awake.6d205314-a834-11ee-ba44-e00af6af6ca6.jpg,0,212,193,509,474
4,awake.6e55e25b-a834-11ee-91f0-e00af6af6ca6 - C...,0,236,194,446,457
...,...,...,...,...,...,...
60,drowsy.8720eaec-a834-11ee-b909-e00af6af6ca6.jpg,2,282,40,590,471
61,drowsy.8856b595-a834-11ee-a806-e00af6af6ca6.jpg,2,172,66,502,472
62,drowsy.9950fe22-a834-11ee-a591-e00af6af6ca6.jpg,0,216,81,478,455
63,drowsy.94769c9b-a834-11ee-b822-e00af6af6ca6.jpg,0,193,86,459,457


In [323]:
train_data = CusData(df=test_df,indices=range(test_df["filename"].unique().shape[0]))

def custom_collate(data):
    return data

train_dl = DataLoader(dataset=train_data,
                      shuffle=True,
                      batch_size=1,
                      collate_fn=custom_collate,
                      pin_memory=True)

In [324]:
def custom_collate(data):
    return data

In [325]:
num_classes = 3
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=num_classes)

In [326]:
device = 'cuda'

In [327]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
num_epoch = 10

In [328]:
model.to(device=device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [ ]:
model()

In [329]:

for epoche in range(num_epoch):
    model.train()
    epoche_loss = 0
    for data in train_dl:
        imgs = []
        targets = []
        for d in data:
            imgs.append(d[0].to(device).type(torch.float32))
            targ = {}
            targ["boxes"] = d[1]["boxes"].to(device).type(torch.float32)
            targ["labels"] = d[1]["labels"].to(device).type(torch.int64)
            targets.append(targ)


        
        loss_dict = model(imgs , targets)
        loss = sum(v for v in loss_dict.values())
        epoche_loss +=  loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print("Epoch: {epoche}  | Loss: {epoche_loss} \n ------------")

0


IndexError: too many indices for tensor of dimension 0